In [ ]:
import os
import numpy as np
import pandas as pd
import re

def process_asc_file(filepath):
    with open(filepath, "r", encoding="utf-8", errors="ignore") as f:
        lines = f.readlines()

    all_beta = []
    all_alpha = []
    all_intensity = []

    alpha_step = 5
    alpha_values = np.arange(0, 360, alpha_step)

    i = 0
    while i < len(lines):
        line = lines[i].strip()

        if line.startswith("*PF_AANGLE"):
            beta = float(line.split("=")[-1])
            i += 1

            while i < len(lines) and not lines[i].strip().startswith("*COUNT"):
                i += 1
            if i >= len(lines):
                break

            count = int(lines[i].strip().split("=")[-1])
            i += 1

            data_block = []
            while i < len(lines) and not lines[i].strip().startswith("*END"):
                row = re.split(r"[\s,]+", lines[i].strip())
                data_block.extend([float(x) for x in row if x])
                i += 1

            data_block = data_block[:len(alpha_values)]
            if len(data_block) != len(alpha_values):
                print(f"⚠️ 文件 {os.path.basename(filepath)} 的 PF_AANGLE = {beta} 数据长度不符，跳过。")
                continue

            all_beta.extend([beta] * len(alpha_values))
            all_alpha.extend(alpha_values)
            all_intensity.extend(data_block)
        else:
            i += 1

    # 计算 θ = 90 - β
    theta_values = 90 - np.array(all_beta)         #asc默认是从

    df = pd.DataFrame({
        "Theta": theta_values,
        "Alpha": all_alpha,
        "Intensity": all_intensity
    })

    return df

# ✅ 设置文件夹路径（将所有 .asc 放在这个文件夹）
asc_folder = "/Users/maguag/Desktop/2e/未命名文件夹"  # ← 修改为你实际的文件夹路径
output_folder = "/Users/maguag/Desktop/2e/未命名文件夹"
os.makedirs(output_folder, exist_ok=True)

# ✅ 遍历所有 asc 文件
for filename in os.listdir(asc_folder):
    if filename.lower().endswith(".asc"):
        filepath = os.path.join(asc_folder, filename)
        df = process_asc_file(filepath)

        output_name = os.path.splitext(filename)[0] + "_converted.txt"
        output_path = os.path.join(output_folder, output_name)
        df.to_csv(output_path, sep="\t", index=False)
        print(f"✅ 已处理 {filename}，输出至 {output_name}")

print("🎉 所有极图文件处理完毕！")